In [2]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import xarray as xr

In [3]:
RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)
az.style.use("arviz-darkgrid")

In [4]:
data_kwargs = dict(sep="\t", names=["userid", "itemid", "rating", "timestamp"])
try:
    data = pd.read_csv("../../dataset/ml-100k/u.data", **data_kwargs)
except FileNotFoundError:
    data = pd.read_csv(pm.get_data("../../dataset/ml-100k/u.data"), **data_kwargs)

data.head()

,userid,itemid,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [5]:
# fmt: off
movie_columns  = ['movie id', 'movie title', 'release date', 'video release date', 'IMDb URL', 
                  'unknown','Action','Adventure', 'Animation',"Children's", 'Comedy', 'Crime',
                  'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
                  'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
# fmt: on

item_kwargs = dict(sep="|", names=movie_columns, index_col="movie id", parse_dates=["release date"])
try:
    movies = pd.read_csv("../../dataset/ml-100k/u.item", **item_kwargs, encoding='latin-1')
except FileNotFoundError:
    movies = pd.read_csv(pm.get_data("../../dataset/ml-100k/u.item"), **item_kwargs)

movies.head()

,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
num_users = data.userid.unique().shape[0]
num_items = data.itemid.unique().shape[0]
sparsity = 1 - len(data) / (num_users * num_items)
print(f"Users: {num_users}\nMovies: {num_items}\nSparsity: {sparsity}")

dense_data = data.pivot(index="userid", columns="itemid", values="rating").values

Users: 943
Movies: 1682
Sparsity: 0.9369533063577546


In [14]:
import numpy as np
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error

def bayesian_dcg(predicted_scores, relevance_scores, k):
        sorted_indices = np.argsort(predicted_scores)[::-1]
        sorted_relevance_scores = relevance_scores[sorted_indices[:k]]
                
        dcg_value = np.sum(sorted_relevance_scores / np.log2(np.arange(2, len(sorted_relevance_scores) + 2)))
        return dcg_value

def bayesian_ndcg(predicted_scores, relevance_scores, k):
    predicted_scores = np.ravel(predicted_scores)
    relevance_scores = np.ravel(relevance_scores)
    
    k = min(k, len(predicted_scores))
    
    mask = ~np.isnan(relevance_scores)
    predicted_scores = predicted_scores[mask]
    relevance_scores = relevance_scores[mask]
    
    ideal_relevance_scores = np.sort(relevance_scores)[::-1][:k]
    
    idcg = bayesian_dcg(ideal_relevance_scores, ideal_relevance_scores, k)
    
    dcg = bayesian_dcg(predicted_scores, relevance_scores, k)

    return dcg / idcg if idcg > 0 else 0.0

def compute_ndcg_for_all_users(predicted_scores, relevance_scores, k):
    ndcg_sum = 0.0
    num_users = predicted_scores.shape[0]

    for i in range(num_users):
        ndcg_sum += bayesian_ndcg(predicted_scores[i], relevance_scores[i], k)

    return ndcg_sum / num_users

def split_train_test(data, percent_test=0.1):
    """Split the data into train/test sets.
    :param int percent_test: Percentage of data to use for testing. Default 10.
    """
    n, m = data.shape  # # users, # movies
    N = n * m  # # cells in matrix

    # Prepare train/test ndarrays.
    train = data.copy()
    test = np.ones(data.shape) * np.nan

    # Draw random sample of training data to use for testing.
    tosample = np.where(~np.isnan(train))  # ignore nan values in data
    idx_pairs = list(zip(tosample[0], tosample[1]))  # tuples of row/col index pairs

    test_size = int(len(idx_pairs) * percent_test)  # use 10% of data as test set
    train_size = len(idx_pairs) - test_size  # and remainder for training

    indices = np.arange(len(idx_pairs))  # indices of index pairs
    sample = rng.choice(indices, replace=False, size=test_size)

    # Transfer random sample from train set to test set.
    for idx in sample:
        idx_pair = idx_pairs[idx]
        test[idx_pair] = train[idx_pair]  # transfer to test set
        train[idx_pair] = np.nan  # remove from train set

    # Verify everything worked properly
    assert train_size == N - np.isnan(train).sum()
    assert test_size == N - np.isnan(test).sum()

    print(train.shape, test.shape)

    # Return train set and test set
    return train, test

def rmse(test_data, predicted):
    """Calculate root mean squared error.
    Ignoring missing values in the test data.
    """
    I = ~np.isnan(test_data)  # indicator for missing values
    N = I.sum()  # number of non-missing values
    sqerror = abs(test_data - predicted) ** 2  # squared error array

    mse = sqerror[I].sum() / N  # mean squared error
    return np.sqrt(mse)  # RMSE


def nmf_with_missing_values(data, n_components, max_iter=20):
    # Initialize with zeros for missing values
    nan_mask = np.isnan(data)
    data_filled = np.nan_to_num(data, nan=0)

    for _ in range(max_iter):
        # Perform NMF
        model = NMF(n_components=n_components, init='random', random_state=0)
        W = model.fit_transform(data_filled)
        H = model.components_

        # Update missing values with the approximation
        approx = W @ H
        data_filled[nan_mask] = approx[nan_mask]

    return W, H, data_filled

train, test = split_train_test(dense_data, percent_test=0.1)

# Train NMF on the training set
n_components = 5
W, H, reconstructed_train = nmf_with_missing_values(train, n_components)

# Reconstruct the test data
reconstructed_data = W @ H

# Evaluate the model's performance
test_rmse = rmse(test, reconstructed_data)
test_ndcg2 = compute_ndcg_for_all_users(reconstructed_data, dense_data, 2)
test_ndcg3 = compute_ndcg_for_all_users(reconstructed_data, dense_data, 3)
test_ndcg5 = compute_ndcg_for_all_users(reconstructed_data, dense_data, 5)

# # Print results
# print("Original Data with Missing Values:")
# print(dense_data)
# print("\nTraining Data (With NaNs):")
# print(train)
# print("\nReconstructed Data:")
# print(reconstructed_data)
print("\nRMSE on Test Data:", test_rmse)
print("NDCG@2 on Test Data:", test_ndcg2)
print("NDCG@3 on Test Data:", test_ndcg3)
print("NDCG@5 on Test Data:", test_ndcg5)

(943, 1682) (943, 1682)


/home/gegen07/micromamba/envs/pymc/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1741: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(



RMSE on Test Data: 1.1582287619327767
NDCG@2 on Test Data: 0.8754355502589832
NDCG@3 on Test Data: 0.8733627225514499
NDCG@5 on Test Data: 0.8697292870435156
